<a href="https://colab.research.google.com/github/deema1999/Tweets_writers_age_classification/blob/main/Tweet_writer_age_classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import xml.etree.cElementTree as et
import re
import os
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# copy images
!mkdir -p Dataset
!cp -rf /content/gdrive/MyDrive/ML/pan2015 Dataset
!cp -rf /content/gdrive/MyDrive/ML/truth.txt Dataset

In [ ]:
#!unzip /content/Project_Data.zip -d Project_Data

# Play with data

In [ ]:
from textblob import TextBlob
data = "His sellection is bery antresting"
output = TextBlob(data).correct()
output

In [ ]:
documents = {}
ind = -1
from textblob import TextBlob

for filename in os.listdir('/content/Dataset/pan2015'):
    full_path = os.path.join('/content/Dataset/pan2015', filename)
    tree = et.parse(full_path)
    root = tree.getroot()
    id = root.attrib['id']
    document = "" 
    ind+=1
    for doc in root.iter('document'):  
      text = doc.text
      text = text.lower()
      text = re.sub(r'(.)\1+', r'\1\1', text)
      # Removing URL's
      # text = re.sub(r'http\S+', '', text)
      # Removing puctuations
      # text = re.sub(r'[^\w\s]', '', text)
      # Removing numbers
      # text = re.sub(r'[\d-]', '', text)
      # Removing multiple spaces
      #text = re.sub(r'\s\s+', '', text)
      # Removing redundent charachters
      #text = re.sub(r'([a-z]+?)\1+', r'\1',text)
      document = document + text + ''
    documents[ind] = [id,document]


In [ ]:
df = pd.DataFrame.from_dict(documents , orient='index' , columns=['Id','Tweets'])
df[df['Id'] == "703806e5-f04b-4e7c-8456-7340784ecf76"]['Tweets']

In [ ]:
#input file
fin = open("/content/Dataset/truth.txt", "rt")
#output file to write the result to
fout = open("out.txt", "wt")
#for each line in the input file
for line in fin:
	#read replace the string and write to output file
	fout.write(line.replace(':::', ','))
#close input and output files
fin.close()
fout.close()

In [ ]:

def prepend_line(file_name, line):
    """ Insert given string as a new line at the beginning of a file """
    # define name of temporary dummy file
    dummy_file = file_name + '.txt'
    # open original file in read mode and dummy file in write mode
    with open(file_name, 'r') as read_obj, open(dummy_file, 'w') as write_obj:
        # Write given line to the dummy file
        write_obj.write(line + '\n')
        # Read lines from original file one by one and append them to the dummy file
        for line in read_obj:
            write_obj.write(line)
    # remove original file
    os.remove(file_name)
    # Rename dummy file as the original file
    os.rename(dummy_file, file_name)

prepend_line("out.txt", "Id,Sex,Age,V,X,Y,Z,W")

In [ ]:
truth = pd.read_csv("out.txt")
# storing this dataframe in a csv file
truth.to_csv('out.csv', index = None)
truth.head()

In [ ]:
data = pd.merge(df, truth, on="Id")
data.size

In [ ]:
data['Age'].unique()

In [ ]:
!pip install demoji

In [ ]:
import demoji
data_copy = data.copy()
target_columns = ['Tweets','Age']
data_copy = data_copy[target_columns]
data_copy.head()
t = data_copy['Tweets'][5]
demoji.findall(t)

In [ ]:
# Explore data
print(data_copy['Age'].unique())
print("All types : ",list(set(data_copy.dtypes.tolist())))

In [ ]:
# To check missing values
print('Sum : \n',data_copy.isnull().sum())
sns.heatmap(data_copy.isnull(), cbar=False, yticklabels=False, cmap="viridis")

In [ ]:
# Dropping the duplicate rows
duplicate_rows = data_copy[data_copy.duplicated()]
print("number of duplicate rows : ",duplicate_rows.shape)

data_copy.drop_duplicates(inplace=True)
print("number of duplicate rows : ",data_copy[data_copy.duplicated()].shape)

In [ ]:
# data visualization
sns.countplot(data_copy['Age'])

In [ ]:
data_copy["Age"] = data_copy["Age"].map({"18-24":1, "25-34":int(2), "35-49":int(3), "50-XX":int(4)})
print(data_copy.head())

# Models Implementation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn.metrics import accuracy_score

Y = data_copy["Age"].values
# split the dataset into training data and test data
from sklearn.model_selection import train_test_split
X_train_NB, X_test_NB, Y_train_NB, Y_test_NB = train_test_split(data_copy, Y, 
                        test_size= 0.20, random_state=100, stratify=Y) # 0.35 ==> SVM : 76

vec = CountVectorizer()
train_vectors = vec.fit_transform(X_train_NB['Tweets'])
test_vectors = vec.transform(X_test_NB['Tweets'])

print("train data : ", train_vectors.shape)
print("test data : ", test_vectors.shape)

# Naive Bayes

In [ ]:
# Classify using Naive Bayes 
# with tfidf : 0.58 , with countVec 0.71 || First method when combining user's tweets
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
MNB_clf = MultinomialNB(alpha=0.5)

# fit model no training data
MNB_clf.fit(train_vectors, Y_train_NB)

# making predictions on the testing set 
predicted = MNB_clf.predict(test_vectors)

# Classification report
from sklearn import metrics
print("Classification report : \n", MNB_clf,"\n", 
      metrics.classification_report(Y_test_NB, predicted))
disp = metrics.plot_confusion_matrix(MNB_clf, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)

print("--------------------------------------")
accuracies = cross_val_score(estimator = MNB_clf, X = train_vectors, y = Y_train_NB, cv = 10)
print("\nAccuracy with 10-kfold: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

# Logistic Reg

In [ ]:
# Classify using Logistic Regression #0.71
# with tfidf : 0.71 , with countVec 0.84
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=10, random_state=42)

# fit model no training data
logisticRegr.fit(train_vectors, Y_train_NB)

# making predictions on the testing set 
predicted = logisticRegr.predict(test_vectors)

# Classification report
from sklearn import metrics
print("Classification report : \n", logisticRegr,"\n", 
      metrics.classification_report(Y_test_NB, predicted))
disp = metrics.plot_confusion_matrix(logisticRegr, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)

print("------------------------------------------------")
accuracies = cross_val_score(estimator = logisticRegr, X = train_vectors, y = Y_train_NB, cv = 10)
print("\nAccuracy with 10-kfold: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

# SVM

In [ ]:
# Classify using SVM - 0.77 after tuning 0.81 by using C=10
# with tfidf : 0.81 , with countVec 0.81
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import pickle
# all parameters not specified are set to their defaults
svmClassifier = SVC(kernel="rbf", C=10, gamma=0.0001)
#svm_classifier = SVC()


#svm_classifier = SVC(kernel="poly", degree=5, C=10, gamma=0.01)
#svm_classifier = SVC(kernel="sigmoid", C=10, gamma=0.001)
#svm_classifier = SVC(kernel="rbf", C=300, gamma=0.001)

# fit model no training data
svmClassifier.fit(train_vectors, Y_train_NB)

# making predictions on the testing set 
predicted = svmClassifier.predict(test_vectors)

# Classification report
from sklearn import metrics
print("Classification report : \n", svmClassifier,"\n", 
      metrics.classification_report(Y_test_NB, predicted))
disp = metrics.plot_confusion_matrix(svmClassifier, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)

'''
print("------------------------------------------------")
accuracies = cross_val_score(estimator = svmClassifier, X = train_vectors, y = Y_train_NB, cv = 10)
print("\nAccuracy with 10-kfold: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))'''
pickle.dump(svmClassifier, open("svc_model", 'wb'))

In [ ]:
loaded_model = pickle.load(open("svc_model", 'rb'))
result = loaded_model.score(test_vectors, Y_test_NB)
print(result)

#Classification report
from sklearn import metrics
disp = metrics.plot_confusion_matrix(svmClassifier, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)

# KNN

In [ ]:
# Classify using KNN 0.65
# with tfidf : 0.65 , with countVec 0.52

from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier 

# Create the classifier
"""
algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
Algorithm used to compute the nearest neighbors:

‘ball_tree’ will use BallTree

‘kd_tree’ will use KDTree

‘brute’ will use a brute-force search.

‘auto’ will attempt to decide the most appropriate algorithm based on the 
values passed to fit method.
"""
knn_cls = KNeighborsClassifier(n_neighbors = 10) # 77%

# Train the classifier
knn_cls.fit(train_vectors,Y_train_NB)

# Predict the value of X_test
predicted = knn_cls.predict(test_vectors)

# Classification report
from sklearn import metrics
print("Classifier : ", knn_cls)
print("Classification report for classifier : \n",
      metrics.classification_report(Y_test_NB, predicted))

disp = metrics.plot_confusion_matrix(knn_cls, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)

'''
print("-------------------------------------------")
accuracies = cross_val_score(estimator = knn_cls, X = train_vectors, y = Y_train_NB, cv = 10)
print("\nAccuracy with 10-kfold: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))'''

# SGD

In [ ]:
# Classify using SGD 0.81
# with tfidf : 0.81 , with countVec 0.84

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
# all parameters not specified are set to their defaults
sgdClassifier = SGDClassifier(random_state=42)

# fit model no training data
sgdClassifier.fit(train_vectors, Y_train_NB)

# making predictions on the testing set 
predicted = sgdClassifier.predict(test_vectors)

# Classification report
from sklearn import metrics
print("Classification report : \n", sgdClassifier,"\n", 
      metrics.classification_report(Y_test_NB, predicted))
disp = metrics.plot_confusion_matrix(sgdClassifier, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)

print("-------------------------------------------")
accuracies = cross_val_score(estimator = sgdClassifier, X = train_vectors, y = Y_train_NB, cv = 10)
print("\nAccuracy with 10-kfold: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

# XGBoosting

In [ ]:
# Classify using XGBoosting 0.65
# with tfidf : 0.65 , with countVec 0.71

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
XG_clf = XGBClassifier(random_state=100)

# fit model no training data
XG_clf.fit(train_vectors, Y_train_NB)

# make predictions for test data
y_pred = XG_clf.predict(test_vectors)
predicted = [round(value) for value in y_pred]

# Classification report
from sklearn import metrics
print("Classification report : \n", XG_clf,"\n", 
      metrics.classification_report(Y_test_NB, predicted))
disp = metrics.plot_confusion_matrix(XG_clf, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)

print("------------------------------------------------")
accuracies = cross_val_score(estimator = svmClassifier, X = train_vectors, y = Y_train_NB, cv = 10)
print("\nAccuracy with 10-kfold: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

# MLP

In [ ]:
# Classify using MLPClassifier 0.81(50/100,50)
# with tfidf : 0.81 , with countVec 0.84(100,50) 0.87(100,50,50)

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100,50, ), batch_size=64,
                    solver="adam", alpha=0.005, activation="logistic", 
                    max_iter=50000, random_state=42)


# fit model no training data
mlp.fit(train_vectors, Y_train_NB)

# make predictions for test data
predicted = mlp.predict(test_vectors)

# Classification report
from sklearn import metrics
print("Classification report : \n", mlp,"\n", 
      metrics.classification_report(Y_test_NB, predicted))
disp = metrics.plot_confusion_matrix(mlp, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# training the model on training set

# n_estimators : The number of trees in the forest.
# max_depth : The maximum depth of the tree.
# n_jobsint : The number of jobs to run in parallel
rf_clf = RandomForestClassifier(n_estimators=50, max_depth=10)
rf_clf.fit(train_vectors,Y_train_NB)

# making predictions on the testing set 
predicted = rf_clf.predict(test_vectors)

# comparing actual response values (y_test) with predicted response values (predicted)
print("Classification report : \n", rf_clf,"\n", metrics.classification_report(Y_test_NB, predicted))
disp = metrics.plot_confusion_matrix(rf_clf, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)


# Cross Validation

In [ ]:
# Cross-validation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

Y = data_copy["Age"].values
vec = CountVectorizer()

# split the dataset into training data and test data
from sklearn.model_selection import train_test_split
X_train_NB, X_test_NB, Y_train_NB, Y_test_NB = train_test_split(data_copy, Y, 
                        test_size= 0.20, random_state=100, stratify=Y)


train_vectors = vec.fit_transform(X_train_NB['Tweets'])
test_vectors = vec.transform(X_test_NB['Tweets'])

print("\n K-fold Cross-Validation : *********")
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, random_state=100)
model_2 = SVC(C= 10, gamma=0.0001, kernel='rbf')
results_model_2 = cross_val_score(model_2, train_vectors, Y_train_NB, cv=kfold)
print("Scores   2 :",results_model_2)
print("Accuracy 2 :",results_model_2.mean())
print("std      2 :",np.std(results_model_2))

'''
print("\n Stratified K-fold Cross-Validation : *********")
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits=10, random_state=100)
model_3 = SVC(kernel="linear", C=10.0, gamma= 0.1)
results_model_3 = cross_val_score(model_3, train_vectors, Y_train_NB[:7000], cv=skfold)
print("Scores   3 :",results_model_3)
print("Accuracy 3 :",results_model_3.mean())
print("std      3 :",np.std(results_model_3))'''

# Hyperparameters Tuning

In [ ]:
# Hyperparameters Tuning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

param_grid = [
  {"C": [1, 10, 50, 100, 500, 1000], "kernel": ["linear"]},
  {"C": [1, 10, 100, 1000],"degree":[3, 5, 7],  "gamma": [0.001, 0.0001], "kernel": ["poly"]},
  {"C": [1, 10, 20, 50, 100, 500, 1000, 2000], "gamma": [0.001, 0.0001 , 0.0003], "kernel": ["rbf", "sigmoid"]},
  # {'alpha':[0.0001 ,0.5, 0.001 , 2 , 5 ,20]}
  # {"hidden_layer_sizes":[(100,50, ) ,(100,100)], "batch_size":[64],"solver":["adam"], "alpha":[0.005 , 0.0001], "activation":["logistic"], "max_iter":[50000], "random_state":[42]}
]

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 1) 
grid.fit(train_vectors, Y_train_NB)

print("----------------------------------------------------------------------")
print("The best parameter after tuning :",grid.best_params_)  
print("Our model looks after hyper-parameter tuning",grid.best_estimator_)

predicted = grid.predict(test_vectors)

from sklearn import metrics
print("Classification report : \n", grid,"\n", 
      metrics.classification_report(Y_test_NB, predicted))
disp = metrics.plot_confusion_matrix(grid, test_vectors, Y_test_NB)
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix: \n", disp.confusion_matrix)

# Gradio

In [ ]:
!pip install gradio

In [ ]:
# https://www.gradio.app/ml_examples
# https://www.kaggle.com/scolianni/mnistasjpg

import tensorflow as tf
from tensorflow import keras
import numpy as np
import gradio as gr
import requests

def inference(tweet):
    labels = ['18-24', '25-34', '35-49', '50-XX']
    texts = vec.transform([tweet])
    pred = mlp.predict_proba(texts)
    #pred = np.append(pred,int(not pred[0]))
    dictionary = dict(zip(labels, map(float, pred[0])))
    return dictionary


gr.Interface(fn=inference, 
             inputs="textbox", 
             outputs=gr.outputs.Label(num_top_classes=4)).launch(share=True) #, debug=True Use in Colab


